In [9]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import numpy as np
import re

In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [24]:
df = pd.read_csv('SIAPE_remuneracao_liquida_executivo_sexo_raca.csv')
df

,ano,liquido,sexo,raca
0,1999,"6527,2",Masculino,Branco
1,1999,"4768,7",Masculino,Negro
2,1999,"5261,6",Feminino,Branco
3,1999,4010,Feminino,Negro
4,2000,"5690,3",Masculino,Branco
...,...,...,...,...
83,2019,"5996,2",Feminino,Negro
84,2020,"8774,2",Masculino,Branco
85,2020,"6272,2",Masculino,Negro
86,2020,"7753,8",Feminino,Branco


In [25]:
df.rename(columns={'liquido': 'remuneracao_liquida', 'raca' : 'cor_raca', 'sexo': 'genero'}, inplace= True) #padronizando


In [26]:
df

,ano,remuneracao_liquida,genero,cor_raca
0,1999,"6527,2",Masculino,Branco
1,1999,"4768,7",Masculino,Negro
2,1999,"5261,6",Feminino,Branco
3,1999,4010,Feminino,Negro
4,2000,"5690,3",Masculino,Branco
...,...,...,...,...
83,2019,"5996,2",Feminino,Negro
84,2020,"8774,2",Masculino,Branco
85,2020,"6272,2",Masculino,Negro
86,2020,"7753,8",Feminino,Branco


In [27]:
def transformar(nome):
    nome =re.sub(r"\bFeminino\b", "Mulheres", nome)
    nome =re.sub(r"\bMasculino\b", "Homens", nome)
    return nome

In [28]:
df['genero'] = df['genero'].apply(transformar)


In [29]:
df['genero_raca'] = df['genero'] + ' ' + df['cor_raca'] 

In [30]:
df['genero_raca'].unique()

array(['Homens Branco', 'Homens Negro', 'Mulheres Branco',
       'Mulheres Negro'], dtype=object)

In [31]:
def transforme(nome):
    nome =re.sub(r"\bHomens Branco\b", "Homens Brancos", nome)
    nome =re.sub(r"\bHomens Negro\b", "Homens Negros", nome)
    nome =re.sub(r"\bMulheres Branco\b", "Mulheres Brancas", nome)
    nome =re.sub(r"\bMulheres Negro\b", "Mulheres Negras", nome)
    return nome

In [32]:
df['genero_raca'] = df['genero_raca'].apply(transforme)

In [33]:
df

,ano,remuneracao_liquida,genero,cor_raca,genero_raca
0,1999,"6527,2",Homens,Branco,Homens Brancos
1,1999,"4768,7",Homens,Negro,Homens Negros
2,1999,"5261,6",Mulheres,Branco,Mulheres Brancas
3,1999,4010,Mulheres,Negro,Mulheres Negras
4,2000,"5690,3",Homens,Branco,Homens Brancos
...,...,...,...,...,...
83,2019,"5996,2",Mulheres,Negro,Mulheres Negras
84,2020,"8774,2",Homens,Branco,Homens Brancos
85,2020,"6272,2",Homens,Negro,Homens Negros
86,2020,"7753,8",Mulheres,Branco,Mulheres Brancas


In [34]:
df['remuneracao_liquida'] = df['remuneracao_liquida'].str.replace(',','.').astype(float)


In [35]:
df = df[['ano', 'genero', 'cor_raca', 'genero_raca', 'remuneracao_liquida' ]]
df

,ano,genero,cor_raca,genero_raca,remuneracao_liquida
0,1999,Homens,Branco,Homens Brancos,6527.2
1,1999,Homens,Negro,Homens Negros,4768.7
2,1999,Mulheres,Branco,Mulheres Brancas,5261.6
3,1999,Mulheres,Negro,Mulheres Negras,4010.0
4,2000,Homens,Branco,Homens Brancos,5690.3
...,...,...,...,...,...
83,2019,Mulheres,Negro,Mulheres Negras,5996.2
84,2020,Homens,Branco,Homens Brancos,8774.2
85,2020,Homens,Negro,Homens Negros,6272.2
86,2020,Mulheres,Branco,Mulheres Brancas,7753.8


Subindo pro GBQ

In [36]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [37]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
 bigquery.SchemaField('genero', 'STRING', description= 'Gênero autodeclarado ou não'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('genero_raca', 'STRING', description= 'Gênero e raça da pessoa observada'),
 bigquery.SchemaField('remuneracao_liquida','FLOAT',description='Remuneração liquida da pessoa observada')]

In [38]:
table_ref = dataset_ref.table('SIAPE_remuneracao_liquida_executivo_sexo_raca')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref,job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=9c7cd26b-6e0a-42bd-967d-5b42c3919fff>